In [ ]:
#https://www.youtube.com/watch?v=Lakz2MoHy6o

In [ ]:
import numpy as np
# from scipy import signal

In [ ]:
def correlate2d_valid(input_matrix, kernel):
    # Get the dimensions of the input matrix and the kernel
    input_height, input_width = input_matrix.shape
    kernel_height, kernel_width = kernel.shape

    # Determine the output dimensions
    output_height = input_height - kernel_height + 1
    output_width = input_width - kernel_width + 1

    output = np.zeros((output_height, output_width))

    for i in range (output_height):
        for j in range (output_width):
            output[i,j] = np.sum(input_matrix[i:i+kernel_height, j:j+kernel_width] * kernel)

    return output


def convolve2d_full(input_matrix, kernel):
    # Get the dimensions of the input matrix and the kernel
    input_height, input_width = input_matrix.shape
    kernel_height, kernel_width = kernel.shape

    # Pad the input for full convolution
    pad_height = kernel_height - 1
    pad_width = kernel_width - 1
    padded_input = np.pad(input_matrix, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant')

    kernel = np.rot90(kernel, 2)


    output = correlate2d_valid(padded_input, kernel)

    return output


# Example below

# input_mat = np.array([[3, 0, 1, 2, 7],
#                          [1, 5, 8, 9, 3],
#                          [2, 7, 2, 5, 1],
#                          [0, 1, 3, 1, 6],
#                          [4, 2, 1, 6, 2]], dtype = 'float')
# kernel = np.array([[-1, 0, 1],
#                          [-2, 0, 2],
#                          [-1, 0, 1]], dtype = 'float')

# mat1 = correlate2d_valid(input_mat, kernel, "full")
# print(mat1)

# print()

# mat3 = convolve2d_full(input_mat, kernel)
# print(mat3)

In [ ]:
class Layer:
  def __init__(self):
    self.input=None
    self.output=None

  def forward_prop(self,input):
    pass

  def backward_prop(self,output_gradient,learning_rate):
    pass

In [ ]:
class Convolutional(Layer):
  def __init__(self,input_shape,kernel_size,num_kernels):
    input_channels,input_height,input_width=input_shape #input_shape is a tuple
    self.num_kernels=num_kernels                          #no of kernels/filters
    self.input_shape=input_shape
    self.input_channels=input_channels
    self.output_shape=(num_kernels,input_height-kernel_size+1,input_width-kernel_size+1)
    self.kernels_shape=(num_kernels,input_channels,kernel_size,kernel_size)


    #filter values are initialized randomly with shape (depth, input_depth, kernel_size, kernel_size)
    self.kernels=np.random.randn(*self.kernels_shape)

    #bias is added to each output, initialized randomly with shape (depth, output_height, output_width)
    self.biases=np.random.randn(*self.output_shape)

  def forward_prop(self,inputs):
    self.inputs=inputs
    input_channels, input_height, input_width = self.inputs.shape
    num_kernel, _, kernel_height, kernel_width = self.kernels.shape

    output_height=input_height-kernel_height+1
    output_width=input_width-kernel_width+1

    self.feature_maps=np.copy(self.biases)

    #convolution operation
    for kernel_idx in range(self.num_kernels):
      for channel_idx in range(self.input_channels):
        self.feature_maps[kernel_idx] += correlate2d_valid(self.inputs[channel_idx], self.kernels[kernel_idx, channel_idx])

    return self.feature_maps



    def backward_prop(self,output_gradient,learning_rate):
      kernels_gradient=np.zeros(self.kernels_shape)
      inputs_gradient=np.zeros(self.inputs.shape)

      input_channels,input_height,input_width=self.inputs.shape
      num_kernel,_,kernel_height,kernel_width=self.kernels.shape
      output_height=input_height-kernel_height+1
      output_width=input_width-kernel_width+1


      #compute input and kernel gradient
      for kernel_idx in range(self.num_kernels):
        for channel_idx in range(self.input_channels):
          kernels_gradient[kernel_idx, channel_idx] = correlate2d_valid(self.inputs[channel_idx], output_gradient[kernel_idx])
          inputs_gradient[channel_idx] += convolve2d_full(self.output_gradient[kernel_idx], self.kernels[kernel_idx, channel_idx])


      #updating kernels and biases
      self.kernels-=learning_rate*kernels_gradient
      self.biases-=learning_rate*output_gradient

      return inputs_gradient

In [ ]:
#to reshape the output of the convolutional layer as the dense layer requires input to be a column vector

class Reshape(Layer):
  def __init__(self,input_shape,output_shape):
    self.input_shape=input_shape
    self.output_shape=output_shape

  def forward_prop(self,input):
    return np.reshape(input,self.output_shape)

  def backward_prop(self,output_gradient,learning_rate):
    return np.reshape(output_gradient,self.input_shape)


In [ ]:
def binary_cross_entropy(y_true,y_pred):
  bce_loss=-np.mean(y_true*np.log(y_pred)+(1-y_true)*np.log(1-y_pred))
  return bce_loss

In [ ]:
#derivative of BCE loss function

def D_binary_cross_entropy(y_true,y_pred):
  D=((1-y_true)/(1-y_pred)-y_true/y_pred)/ np.size(y_true)
  return D


In [ ]:
#base class for activation functions

class Activation(Layer):
  def __init__(self,activation,activation_der):
    self.activation=activation
    self.activation_der=activation_der  #derivative of the activation

  def forward_prop(self,input):
    self.input=input
    return self.activation(self.input)

  #computes the gradient of loss wrt input
  def backward_prop(self,output_gradient,learning_rate):
    return np.multiply(output_gradient, self.activation_der(self.input))



In [ ]:

class Sigmoid(Activation):
  def __init__(self):

    def sigmoid(y):
      return 1/(1+ np.exp(-y))

    def sigmoid_der(y):
      s=sigmoid(y)
      return s*(1-s)
    #calls the constructor of the Activation(parent) class
    #passess the sigmoid and its derivative to the Activation class
    super().__init__(sigmoid,sigmoid_der)

In [ ]:
class ReLU(Activation):
  def __init__(self):
    def relu(x):
      return np.maximum(0,x)

    def relu_der(x):
      return np.where(x>0,1,0)   #1 for positive values, 0 for negative

    super().__init__(relu,relu_der)

In [ ]:
class Dense(Layer):
  def __init__(self,input_size,output_size):
    #assign random weights and biases
    #they will be updated during back prop
    self.weights=np.random.randn(output_size,input_size)
    self.bias=np.random.randn(output_size,1)

  def forward_prop(self,input):
    self.input=input
    M=np.dot(self.weights,self.input)+self.bias
    return M

  def backward_prop(self,output_gradient,learning_rate):
     #dot product of output grad and input transpose
    weights_gradient=np.dot(output_gradient,self.input.T)
    inputs_gradient=np.dot(self.weights.T,output_gradient)

    self.weights-=learning_rate*weights_gradient
    self.bias-=learning_rate*output_gradient

    return inputs_gradient


In [ ]:
#functions for training the model and predicting

def predict(model,input):
  output=input

  for layer in model:
    output=layer.forward_prop(output)
    # print("Type of output: ",type(output))

  return output

In [ ]:
from re import X
def train(model,x_train,y_train,loss,loss_der,epochs=1000,alpha=0.01,v=True):

  for e in range(epochs):
    err=0
    #zip pairs each pair of input with its corresponding output
    for x,y in zip(x_train,y_train):
      output=predict(model,x)
      # print("type is: ",type(output))

      #compute the loss for this predicted output
      err=err+loss(y,output)
      #compute the gradient of the loss
      grad=loss_der(y,output)

      #iterate the network in reverse order
      for layer in reversed(model):
        grad=layer.backward_prop(grad,alpha)  #returns the grad wrt input of the layer

    err=err/len(x_train)  #compute the average error for the epoch

    if v:
      print(f"{e+1}/{ epochs}, error={err}")



In [ ]:

from keras.datasets import mnist
from keras.utils import to_categorical

def preprocess_data(x, y, limit):
    zero_index = np.where(y == 0)[0][:limit]
    one_index = np.where(y == 1)[0][:limit]
    all_indices = np.hstack((zero_index, one_index))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    y = to_categorical(y)
    y = y.reshape(len(y), 2, 1)
    return x, y

# load MNIST from server, limit to 100 images per class since we're not training on GPU
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, 100)
x_test, y_test = preprocess_data(x_test, y_test, 100)

# print("Shape",x_train.shape)
# print("type",type(x_train))

# neural network
network = [
    Convolutional((1, 28, 28), 3, 5),
    Sigmoid(),
    Reshape((5, 26, 26), (5 * 26 * 26, 1)),
    Dense(5 * 26 * 26, 100),
    Sigmoid(),
    Dense(100, 2),
    Sigmoid()
]

# train
train(
    network,
    x_train,
    y_train,
    binary_cross_entropy,
    D_binary_cross_entropy,
    epochs=20,
    alpha=0.1
)



1/20, error=0.4308836679645987
2/20, error=0.12005237344073479
3/20, error=0.09095241957927268
4/20, error=0.07788493535422468
5/20, error=0.044867453761574
6/20, error=0.030000899038268292
7/20, error=0.0326328685384639
8/20, error=0.023800431049711107
9/20, error=0.031320266510846594
10/20, error=0.02789341586784317
11/20, error=0.012122814773818902
12/20, error=0.013157293139597686
13/20, error=0.010363976513047324
14/20, error=0.007165678006629313
15/20, error=0.007134751635606503
16/20, error=0.0061012266617984446
17/20, error=0.005059775107213872
18/20, error=0.004599319544875391
19/20, error=0.004248824096376183
20/20, error=0.003900418529766483


In [ ]:
# test
correct = 0
total= 0
for x, y in zip(x_test, y_test):
    output = predict(network, x)
    # print(f"pred: {np.argmax(output)}, true: {np.argmax(y)}")
    total += 1
    if (np.argmax(output) == np.argmax(y)):
        correct += 1

print(f"Accuracy: {correct / total * 100}%")

Accuracy: 99.5%
